## Adapted Unet
In the this notebook is the pipeline to train and create submission with an adapted UNet. It uses padding to predict an image of the same size than the input. We made the claim than since it's only based on convolution, the size doesn't matter much. So we predict the submission images in the same way.

Training: (3x400x400) => (400x400)  
Submission: (3x608x608) => (608x608)  

In [1]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

import sys
sys.path.insert(0, '../')

from Adapted_UNet import *
from images_helpers import *
from unet_helpers import *

### Load data, create training and testing dataset

In [2]:
# Training dataset
train_imgs,_ = load_images(UNET_PREFIX + AUGMENTED_IMAGE_TRAIN_DIR)
train_gt_imgs,_ = load_images(UNET_PREFIX + AUGMENTED_GT_TRAIN_DIR)

train_img_tensor = images_to_tensor(train_imgs)
train_gt_tensor = gts_to_tensor(train_gt_imgs)

train_dataset = torch.utils.data.TensorDataset(train_img_tensor, train_gt_tensor)

# Test dataset
test_imgs,_ = load_images(UNET_PREFIX + AUGMENTED_IMAGE_TEST_DIR)
test_gt_imgs,_ = load_images(UNET_PREFIX + AUGMENTED_GT_TEST_DIR)

test_img_tensor = images_to_tensor(test_imgs)
test_gt_tensor = gts_to_tensor(test_gt_imgs)

test_dataset = torch.utils.data.TensorDataset(test_img_tensor, test_gt_tensor)

900 images loaded
900 images loaded
100 images loaded
100 images loaded


### Train the unet

In [3]:
# TRAINING PARAMETERS
INIT_LR = 0.001
NUM_EPOCHS = 200
BATCH_SIZE = 4
PROBA_DROPOUT = 0
PROBA_DROPOUT_MIDDLE = 0.3
lossFunc = BCEWithLogitsLoss()
#lossFunc = sigmoid_f1_loss

LOAD_MODEL = False
LOAD_MODEL_NAME = UNET_PREFIX + MODEL_DIR + "unet_epoch_28"
LOAD_OPT_NAME = UNET_PREFIX + MODEL_DIR + "unet_epoch_28_opt"

MODEL_BATCH_NAME = UNET_PREFIX + MODEL_DIR + "unet_batch_"
MODEL_EPOCH_NAME = UNET_PREFIX + MODEL_DIR + "unet_epoch_"

In [4]:
torch.cuda.empty_cache()
unet = UNet(PROBA_DROPOUT, PROBA_DROPOUT_MIDDLE).to(DEVICE)
opt = Adam(unet.parameters(), lr=INIT_LR)
if LOAD_MODEL:
    unet.load_state_dict(torch.load(LOAD_MODEL_NAME))
    opt.load_state_dict(torch.load(LOAD_OPT_NAME))
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

In [5]:
torch.cuda.is_available()

False

In [ ]:
unet, opt, losses, test_losses = train_model(unet, opt, NUM_EPOCHS, train_loader,test_dataset, lossFunc,\
                                             num_testing=80, num_test_per_epoch=7, num_save_per_epoch=7)

### Create submission

In [4]:
import os
import numpy as np

In [5]:
SUBMISSION_PREDICTION_DIR = UNET_PREFIX + "predictions_submission/"
TEST_SET_DIR = UNET_PREFIX + "data/test_set_images/"

test_files = os.listdir(TEST_SET_DIR)
imgs = [load_image(TEST_SET_DIR + f + "/" + f +".png") for f in test_files]
imgs = np.array([np.moveaxis(img, -1, 0) for img in imgs])

In [8]:
model = UNet(PROBA_DROPOUT, PROBA_DROPOUT_MIDDLE).to(DEVICE)
model.load_state_dict(torch.load(UNET_PREFIX + MODEL_DIR + "unet_epoch_28"))

<All keys matched successfully>

In [9]:
def sigmoid(t):
    return np.exp(-np.logaddexp(0, -t))

with torch.no_grad():
    model.eval()
    idx = 0;
    
    files_pred = []
    for i in range(len(imgs)):
        if i % 2 ==0:
            xs = imgs[i:i+2]
            x = torch.tensor(xs).to(DEVICE)
            y_pred = model(x)
            y_pred=y_pred.cpu()
            for j in range(y_pred.shape[0]):
                y_pred_ = sigmoid(y_pred[j,0].detach().numpy()) 
                y_pred_ = np.where(y_pred_ > 0.5, 1, 0)            

                img_pred = from_mask_to_img(y_pred_)
                pred_name = SUBMISSION_PREDICTION_DIR + test_files[idx]+".png"
                img_pred.save(pred_name)
                idx += 1
                files_pred.append(pred_name)
            print(f"Saved {i}")


    masks_to_submission("submissionUnet", *files_pred)

Saved 0
Saved 2
Saved 4
Saved 6
Saved 8
Saved 10
Saved 12
Saved 14
Saved 16
Saved 18
Saved 20
Saved 22
Saved 24
Saved 26
Saved 28
Saved 30
Saved 32
Saved 34
Saved 36
Saved 38
Saved 40
Saved 42
Saved 44
Saved 46
Saved 48
../predictions_submission/test_1.png
../predictions_submission/test_10.png
../predictions_submission/test_11.png
../predictions_submission/test_12.png
../predictions_submission/test_13.png
../predictions_submission/test_14.png
../predictions_submission/test_15.png
../predictions_submission/test_16.png
../predictions_submission/test_17.png
../predictions_submission/test_18.png
../predictions_submission/test_19.png
../predictions_submission/test_2.png
../predictions_submission/test_20.png
../predictions_submission/test_21.png
../predictions_submission/test_22.png
../predictions_submission/test_23.png
../predictions_submission/test_24.png
../predictions_submission/test_25.png
../predictions_submission/test_26.png
../predictions_submission/test_27.png
../predictions_submiss